In [ ]:
import requests
import hashlib
import datetime 
import time
import pandas as pd
from IPython.display import clear_output

def login(mob):
    #Authenticate a beneficiary by Mobile/OTP
    API_ENDPOINT = "https://cdn-api.co-vin.in/api/v2/auth/public/generateOTP"
    data = {
      "mobile": mob
    }
    r = requests.post(url = API_ENDPOINT, json = data)

    #Confirm mobile OTP for authentication
    otp = input('Please Enter OTP: ')
    #Convert the string into bytes to be acceptable by hash function.
    otpEncoded = otp.encode()
    otpSHA256 = hashlib.sha256(otpEncoded)
    #Return the encoded data in hexadecimal format
    otpHex = otpSHA256.hexdigest()
    
    API_ENDPOINT = "https://cdn-api.co-vin.in/api/v2/auth/public/confirmOTP"
    data = {
      "otp": otpHex,
      "txnId": r.json()["txnId"]
    }
    r = requests.post(url = API_ENDPOINT, json = data)
    response = r.json()
    print(response['token'])
    
def states():
    # Get states
    URL = "https://cdn-api.co-vin.in/api/v2/admin/location/states"
    PARAMS = {}
    headers = {'User-Agent':"Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Mobile Safari/537.36"}
    r = requests.get(url = URL, params = PARAMS, headers = headers)
    data = r.json()
    states = pd.DataFrame(data['states'])
    display(states)

def districts():
    # Get list of districts
    state_id = input('Enter the State_Id ').strip()
    URL = "https://cdn-api.co-vin.in/api/v2/admin/location/districts/{}".format(state_id)
    headers = {'User-Agent':"Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Mobile Safari/537.36"}
    r = requests.get(url = URL, headers = headers)
    data = r.json()
    df = pd.DataFrame(data['districts'])
    display(df)

def search_pin_date():
    #Get vaccination sessions by PIN
    pincode = input('Enter the pincode ').strip()
    date = input('Enter the date (e.g. 15-05-2021) ').strip()
    URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/findByPin?pincode={}&date={}".format(pincode,date)
    headers = {'User-Agent':"Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Mobile Safari/537.36"}
    r = requests.get(url = URL, headers = headers)
    data = r.json()
    try:
        df = pd.DataFrame(data['sessions'])
        df = df[['name', 'district_name','fee_type', 'available_capacity', 'available_capacity_dose1','available_capacity_dose2',
                 'fee','min_age_limit', 'vaccine', 'date','pincode','center_id']]
        df = df[(df['min_age_limit'] == 18) & (df['available_capacity'] > 0)]
        display(df)
    except:
        print('Try after sometime.')

def search_district_date():
    # Get vaccination sessions by district

    district_id = input('Enter the district_id ').strip()
    date = input('Enter the date (e.g. 15-05-2021) ').strip()
    URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/findByDistrict?district_id={}&date={}".format(district_id,date)
    headers = {'User-Agent':"Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Mobile Safari/537.36"}
    r = requests.get(url = URL, headers = headers)
    data = r.json()
    try:
        df = pd.DataFrame(data['sessions'])
        df = df[['name', 'district_name', 'fee_type', 'available_capacity', 'available_capacity_dose1','available_capacity_dose2', 
                 'fee','min_age_limit', 'vaccine', 'date','pincode','center_id']]
        df = df[(df['min_age_limit'] == 18) & (df['available_capacity'] > 0)]
        display(df)
    except:
        print('Try after sometime.')

def search_pin(pin):
    # Get vaccination sessions by PIN for 7 days

    pincode = pin
    date = datetime.date.today().strftime('%d-%m-%Y')
    URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByPin?pincode={}&date={}".format(pincode,date)
    headers = {'User-Agent':"Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Mobile Safari/537.36"}
    r = requests.get(url = URL, headers = headers)
    data = r.json()
    try:
        df = pd.json_normalize(data = data['centers'],record_path='sessions',meta=['center_id', 'name', 'address', 'state_name', 'district_name',
                                                                                   'block_name', 'pincode', 'lat', 'long', 'from', 'to', 'fee_type','sessions'])
        df = df[['name','date','pincode', 'available_capacity', 'vaccine','available_capacity_dose1', 'available_capacity_dose2',
                 'center_id','district_name','fee_type', 'min_age_limit']]
        df = df[(df['min_age_limit'] == 18) & (df['available_capacity'] > 0) & (df['available_capacity_dose1'] > 0)]
        display(df)
    except:
        print('Try after sometime.')
        
def search_district(dist):
    # Get vaccination sessions by PIN for 7 days

    district_id = dist
    date = datetime.date.today().strftime('%d-%m-%Y')
    URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}".format(district_id,date)
    headers = {'User-Agent':"Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Mobile Safari/537.36"}
    r = requests.get(url = URL, headers = headers)
    data = r.json()
    try:
        df = pd.json_normalize(data = data['centers'],record_path='sessions',meta=['center_id', 'name', 'address', 'state_name', 'district_name',
                                                                                   'block_name', 'pincode', 'lat', 'long', 'from', 'to', 'fee_type','sessions'])
        df = df[['name','date','pincode', 'available_capacity', 'vaccine','available_capacity_dose1', 'available_capacity_dose2',
                 'center_id','district_name','fee_type', 'min_age_limit']]
        df = df[(df['min_age_limit'] == 18) & (df['available_capacity'] > 0) & (df['available_capacity_dose1'] > 0)]
        display(df)
    except:
        print('Try after sometime.')

In [ ]:
# login()
# states()
# districts()
# search_pin_date()
# search_district_date()# search_pin()

In [ ]:
for i in range(50):
    clear_output(wait=False)
#     148 Shahdara
    print('Shahdara')
    search_district(148)
    time.sleep(1)
#     145 East Delhi
    print('East Delhi')
    search_district(145)
    time.sleep(7)

In [ ]:
search_district(145)

In [ ]:
search_pin(110092)